In [1]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_core.prompts import ChatPromptTemplate
#from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1", temperature=0)



In [9]:
from langchain_core.pydantic_v1 import BaseModel, Field

class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""

    answer: str = Field(description="The answer to the user's question")
    followup_question: str = Field(description="A followup question the user could ask")

model = llm
model_with_tools = model.bind_tools([ResponseFormatter])

ai_msg = model_with_tools.invoke("What is the powerhouse of the cell?")

ai_msg.tool_calls[0]["args"]

{'answer': 'The powerhouse of the cell is the mitochondria.',
 'followup_question': 'What is the function of mitochondria in a cell?'}

In [10]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")

structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

Joke(setup='A cat walks into a music store and asks the owner...', punchline='Why did the cat join a band? Because it wanted to be the purr-cussionist!', rating=8)

In [12]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

In [13]:
from langchain_core.pydantic_v1 import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [14]:
tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-17T06:24:42.388064Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply', 'arguments': {'a': 3, 'b': 12}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 20965061155, 'load_duration': 8349142732, 'prompt_eval_count': 216, 'prompt_eval_duration': 9129096000, 'eval_count': 22, 'eval_duration': 3484607000}, id='run-612baa06-a76b-4ecd-a3d5-e40cce4e25ac-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '8a81208b-3813-47d0-be43-c9aaf452f312', 'type': 'tool_call'}], usage_metadata={'input_tokens': 216, 'output_tokens': 22, 'total_tokens': 238})

In [4]:
from langchain_core.prompts import ChatPromptTemplate
#from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1", temperature=0)

# プロンプトのテンプレート文章を定義
template = """
次の文章に誤字がないか調べて。誤字があれば訂正してください。
{sentences_before_check}
"""

# テンプレート文章にあるチェック対象の単語を変数化
prompt = ChatPromptTemplate(
    input_variables=["sentences_before_check"],
    template=template,
)

# OpenAIのAPIにこのプロンプトを送信するためのチェーンを作成
chain = prompt | llm

# チェーンを実行し、結果を表示
print(chain.invoke("こんんんちわ、永田です。群馬県の山でヒグマに出くわしました。")['text'])

TypeError: Expected mapping type as input to ChatPromptTemplate. Received <class 'str'>.